In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Train_Data.csv')

df.drop(columns = ['SEQN'],inplace=True)
df.shape
df.sample(10)

,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
1407,2.0,2.0,37.3,110.0,2.0,168.0,22.31,Adult
1222,2.0,2.0,24.7,86.0,2.0,140.0,6.69,Adult
1738,2.0,2.0,23.7,93.0,2.0,58.0,5.06,Adult
482,1.0,2.0,23.9,112.0,2.0,178.0,15.90,Adult
1915,1.0,2.0,29.4,118.0,2.0,231.0,9.65,Senior
508,1.0,2.0,20.2,99.0,2.0,129.0,3.40,Senior
220,2.0,2.0,27.0,90.0,2.0,110.0,2.91,Adult
159,1.0,1.0,43.2,99.0,2.0,93.0,21.05,Adult
1088,2.0,2.0,18.3,89.0,2.0,100.0,4.25,Adult
655,2.0,2.0,20.3,103.0,2.0,91.0,8.25,Adult


In [3]:
df.isnull().mean()*100

RIAGENDR     0.915565
PAQ605       0.661241
BMXBMI       0.915565
LBXGLU       0.661241
DIQ010       0.915565
LBXGLT       0.559512
LBXIN        0.457782
age_group    0.712106
dtype: float64

In [4]:
df.fillna(df.median(numeric_only=True),inplace=True)

In [5]:
df.isnull().mean()*100

RIAGENDR     0.000000
PAQ605       0.000000
BMXBMI       0.000000
LBXGLU       0.000000
DIQ010       0.000000
LBXGLT       0.000000
LBXIN        0.000000
age_group    0.712106
dtype: float64

In [6]:
df['age_group'].value_counts(normalize=True)

age_group
Adult     0.839139
Senior    0.160861
Name: proportion, dtype: float64

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced',max_iter=1000)

In [8]:
df['age_group'] = df['age_group'].map({'Adult':0,'Senior':1})


X = df.drop('age_group',axis=1)
y = df['age_group']


combined = pd.concat([X,y],axis=1)

combined =combined.dropna(subset=['age_group'])

X = combined.drop('age_group',axis=1)
y = combined['age_group']

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=40)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[223 102]
 [ 21  45]]
              precision    recall  f1-score   support

         0.0       0.91      0.69      0.78       325
         1.0       0.31      0.68      0.42        66

    accuracy                           0.69       391
   macro avg       0.61      0.68      0.60       391
weighted avg       0.81      0.69      0.72       391



In [14]:
test_df = pd.read_csv('Test_Data.csv')
test_df.drop(columns = ['SEQN'],inplace=True)
test_df.fillna(df.median(numeric_only=True),inplace=True)

In [15]:
predictions = model.predict(test_df).astype(int)

output_df = pd.DataFrame({'age_group': predictions})

output_df.to_csv('submission1.csv', index=False)